In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


In [ ]:
#### 노트북 셀 가로 넓게 보기
from IPython.core.display import display, HTML

notebook_config = ""
notebook_config += "<style>"
notebook_config += "  .container { width:90% !important; }"
notebook_config += "</style>"
HTML(notebook_config)

In [ ]:
# SEED 및 프로그램 버전 설정
IS_COLAB = True
SEED = 42
PROGRAM_VERSION = "13"

In [ ]:
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings("ignore")
import shutil
import time
from scipy.stats import gmean
from itertools import combinations
from tqdm import tqdm
tqdm.pandas()

import tensorflow as tf
from tensorflow import keras


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import TimeDistributed, LayerNormalization
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import PassiveAggressiveRegressor


from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

# AdaBoost
from sklearn.ensemble import AdaBoostRegressor

# XGBoost
from xgboost import plot_importance
from xgboost import XGBRegressor

# lightgbm
from lightgbm import LGBMRegressor

# catboost
from catboost import CatBoostRegressor

# metric
from sklearn.metrics import mean_absolute_error

from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import RandomizedSearchCV
# from bayes_opt import BayesianOptimization
# from skopt.space import Real, Categorical, Integer
# from skopt import BayesSearchCV


print(tf.__version__)
print(keras.__version__)

2.12.0
2.12.0


In [ ]:
!pip install rdkit mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.6 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=d64daa0e0978e4dcc45dbe0e19a0123c127b1536dcc64c3bd2498c202b338527
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.ipython_useSVG = True
# IPythonConsole.molSize = 500, 500

In [ ]:
# 일단 colab 에서 실행하는 것으로 설정
DATA_DIR = '/content/drive/MyDrive/DACON/bio/new/'
SUBMISSION_DIR = '/content/drive/MyDrive/DACON/bio/sub/'
CLEANED_DIR = '/content/drive/MyDrive/DACON/bio/new/'
MODEL_DIR = '/content/drive/MyDrive/DACON/bio/model/'


In [ ]:
# # colab이 아니면...로컬에서 실행하는 경우면
# if not IS_COLAB:
#     DATA_DIR = './dat/'
#     SUBMISSION_DIR = './sub/'
#     CLEANED_DIR = './cleaned/'
#     MODEL_DIR = './model/'

In [ ]:
train_original = pd.read_csv(f'{DATA_DIR}train.csv')
# dev = pd.read_csv(f'{DATA_DIR}dev.csv')
test = pd.read_csv(f'{DATA_DIR}test.csv')
submission = pd.read_csv(f'{DATA_DIR}sample_submission.csv')

train = train_original # pd.concat([train_original, dev], axis=0)

train_original.shape, train.shape, test.shape

((3498, 11), (3498, 11), (483, 9))

In [ ]:
calc = Calculator(descriptors, ignore_3D=False)
len(calc.descriptors)

1826

In [ ]:
x_train = calc.pandas([Chem.MolFromSmiles(x) for x in train.SMILES])
x_train

100%|██████████| 3498/3498 [21:08<00:00,  2.76it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,21.379612,17.449011,0,0,35.689316,2.421199,4.745523,35.689316,1.274618,4.249154,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,16.539255,14.049653,0,0,26.575899,2.426398,4.757199,26.575899,1.265519,3.984419,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,17.475469,13.660693,2,1,29.802128,2.510668,4.982923,29.802128,1.354642,4.049690,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,27.857311,20.034364,0,1,45.884166,2.532483,4.973440,45.884166,1.310976,4.500758,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,15.722758,12.817176,0,0,26.308663,2.452930,4.905860,26.308663,1.315433,3.935574,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,19.701948,17.330258,0,0,30.902711,2.515832,4.792938,30.902711,1.236108,4.158311,...,10.208580,77.363487,395.052750,10.129558,1615,38,136.0,162.0,10.062500,5.250000
3494,21.148902,17.245220,0,0,35.887372,2.494190,4.985907,35.887372,1.329162,4.237338,...,10.291162,75.955433,359.138225,8.162232,1765,45,144.0,173.0,7.750000,6.027778
3495,14.348985,12.232287,0,0,23.546531,2.450594,4.723643,23.546531,1.239291,3.855798,...,9.677277,66.189153,261.147727,6.872309,795,26,94.0,108.0,6.916667,4.333333
3496,14.458374,12.578196,0,0,23.936088,2.314213,4.623104,23.936088,1.259794,3.855415,...,9.604475,65.335399,284.056385,8.876762,812,25,94.0,107.0,6.916667,4.250000


In [ ]:
train_complete = pd.concat([train, x_train], axis=1)
train_complete

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,...,10.208580,77.363487,395.052750,10.129558,1615,38,136.0,162.0,10.062500,5.250000
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,...,10.291162,75.955433,359.138225,8.162232,1765,45,144.0,173.0,7.750000,6.027778
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,...,9.677277,66.189153,261.147727,6.872309,795,26,94.0,108.0,6.916667,4.333333
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,...,9.604475,65.335399,284.056385,8.876762,812,25,94.0,107.0,6.916667,4.250000


In [ ]:
x_test = calc.pandas([Chem.MolFromSmiles(x) for x in test.SMILES])
x_test

100%|██████████| 483/483 [02:54<00:00,  2.77it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,19.229782,15.428567,0,1,32.037339,2.392367,4.778964,32.037339,1.281494,4.137716,...,9.935713,72.422574,361.193631,6.946031,1607,34,126.0,144.0,7.527778,5.638889
1,21.229782,16.881776,0,0,35.824667,2.462328,4.912724,35.824667,1.326840,4.240550,...,10.330421,76.111616,370.152872,7.554140,1853,45,146.0,175.0,7.840278,5.972222
2,20.155669,15.481463,0,3,32.761443,2.378577,4.757154,32.761443,1.260056,4.175318,...,10.008163,60.614034,347.174610,7.386694,1905,37,132.0,150.0,8.388889,5.722222
3,21.079953,16.076404,0,0,35.577061,2.496055,4.855755,35.577061,1.368349,4.227377,...,10.288444,77.464779,345.133808,8.417898,1673,42,146.0,177.0,6.000000,5.583333
4,20.620135,16.065299,0,1,34.235212,2.546495,4.990622,34.235212,1.316739,4.210465,...,10.380312,75.874629,353.185175,7.207861,1656,47,144.0,176.0,8.722222,5.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,16.291966,14.728760,0,0,28.248662,2.381151,4.715751,28.248662,1.284030,3.995519,...,9.862509,69.656439,306.230728,5.889052,1138,32,108.0,125.0,7.756944,5.222222
479,18.777305,15.822051,0,1,29.983837,2.559820,5.003321,29.983837,1.249327,4.116090,...,10.288852,73.647870,335.184506,6.840500,1280,42,130.0,157.0,9.201389,5.180556
480,20.648645,16.775304,0,0,33.716988,2.612365,5.113784,33.716988,1.296807,4.210713,...,10.447235,76.277596,349.142641,7.758725,1469,48,144.0,177.0,8.722222,5.611111
481,15.343821,13.245018,0,0,24.141151,2.379239,4.758478,24.141151,1.207058,3.909645,...,9.874728,53.565348,339.985883,11.332863,826,31,102.0,118.0,8.388889,4.361111


In [ ]:
test_complete = pd.concat([test, x_test], axis=1)
test_complete

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76,19.229782,...,9.935713,72.422574,361.193631,6.946031,1607,34,126.0,144.0,7.527778,5.638889
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31,21.229782,...,10.330421,76.111616,370.152872,7.554140,1853,45,146.0,175.0,7.840278,5.972222
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86,20.155669,...,10.008163,60.614034,347.174610,7.386694,1905,37,132.0,150.0,8.388889,5.722222
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21,21.079953,...,10.288444,77.464779,345.133808,8.417898,1673,42,146.0,177.0,6.000000,5.583333
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15,20.620135,...,10.380312,75.874629,353.185175,7.207861,1656,47,144.0,176.0,8.722222,5.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,4.207,306.443,2,1,7,4.207,55.13,16.291966,...,9.862509,69.656439,306.230728,5.889052,1138,32,108.0,125.0,7.756944,5.222222
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,-0.608,335.398,5,0,1,-1.736,70.16,18.777305,...,10.288852,73.647870,335.184506,6.840500,1280,42,130.0,157.0,9.201389,5.180556
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,1.792,349.383,3,1,3,1.792,69.72,20.648645,...,10.447235,76.277596,349.142641,7.758725,1469,48,144.0,177.0,8.722222,5.611111
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0.790,341.132,3,2,2,0.423,69.64,15.343821,...,9.874728,53.565348,339.985883,11.332863,826,31,102.0,118.0,8.388889,4.361111


In [ ]:
train_complete.to_csv(f"{CLEANED_DIR}train_clean.csv", index=False)
test_complete.to_csv(f"{CLEANED_DIR}test_clean.csv", index=False)
